# PSI Numerical Methods 2024 - Homework Assignment on Model Fitting & MCMC

We're going to put together everything we have learned so far to re-do the data analysis for the
Perlmutter et al. 1999 paper on the discovery of dark energy!  (https://ui.adsabs.harvard.edu/abs/1999ApJ...517..565P/abstract)

Start by `Forking` this repository on Github: https://github.com/dstndstn/PSI-Numerical-Methods-2024-MCMC-Homework
And then clone the repository to your laptop or to Symmetry.
You can modify this notebook, and when you are done, save it, and then `git commit -a` the results,
and `git push` them back to your fork of the repository.  You will "hand in" your homework by giving
a link to your Github repository, where the marker will be able to read your notebook.

First, a little bit of background on the cosmology and astrophysics.  The paper reports measurements
of a group of supernova explosions of a specific type, "Type 1a".  These are thought to be caused by
a white dwarf star that has a companion star that "donates" gas to the white dwarf.  It gradually gains
mass until it exceeds the Chandresekhar mass, and explodes.  Since they all explode through the same
mechanism, and with the same mass, they should all have the same intrinsic brightess.  It turns out to
be a _little_ more complicated than that, but in the end, these Type-1a supernovae can be turned into
"standard candles", objects that are all the same brightness.  If you can also measure the redshift of
each galaxy containing the supernova, then you can map out this brightness--redshift relation, and the
shape of that relation depends on how the universe grows over cosmic time.  In turn, the growth rate of
the universe depends on the contents of the universe!

In this way, these Type-1a supernova allow us to constrain the parameters of a model of the universe.
Specifically, the model is called "Lambda-CDM", a universe containing dark energy and matter (cold dark matter,
plus regular matter).  We will consider a two-parameter version of this model: $\Omega_M$, the
amount of matter, and $\Omega_{\Lambda}$, the amount of dark energy.  These are in cosmology units of
"energy density now relative to the critical density", where the critical density is the energy density you need
for the universe to be spatially flat (angles of a large triangle sum to 180 degrees).
So $\Omega_M = 1$, $\Omega_{\Lambda} = 0$ would be a flat universe containing all matter, while
$\Omega_M = 0.25$, $\Omega_{\Lambda} = 0.5$ would be a spatially closed universe with dark energy and matter.
Varying these ingredients changes the growth history of the universe, which changes how much the light from a
supernova is redshifted, and how its brightness drops off with distance.

(In the code below, we will call these `Omega_M` = $\Omega_M$ and `Omega_DE` = $\Omega_{\Lambda}$.)

Distance measurements in cosmology are complicated -- see https://arxiv.org/abs/astro-ph/9905116 for details!
For this assignment, we will use a cosmology package that will handle all this for us.  All we need to use is
the "luminosity distance", which is the one that tells you how objects get fainter given a redshift.

In [ ]:
# Let's start by installing the Cosmology package!
using Pkg
Pkg.add("GLMakie")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Cosmology")
Pkg.add("Statistics")
Pkg.add("FHist")

In [ ]:
# We'll also end up using all our old friends:
using GLMakie
using CSV
using DataFrames
using Cosmology
using Optim
using Statistics
using FHist

In [ ]:
# There is a data file in this directory, taken basically straight out of the Perlmutter+1999 paper.  We can read it with the CSV package.
data = CSV.read("p99-data.txt", DataFrame, delim=" ", ignorerepeated=true);
println(data[1:3, :])

In [ ]:
# Make a copy of the data columns that we want to treat as the "y" measurements.
# These are the measured brightnesses, and their Gaussian uncertainties (standard deviations).
data.mag = data.m_b_eff
data.sigma_mag = data.sigma_m_b_eff;

In [ ]:
# f = Figure()
# Axis(f[1, 1], title="Perlmutter+99_Supernovae", xlabel="Redshift_z", ylabel="m_B")
# errorbars!(data.z, data.mag, data.sigma_mag)
# scatter!(data.z, data.mag, markersize=5, color=:maroon)
# f

In [ ]:
# Here is how we will use the "cosmology" package.  This will create a cosmology "object" with the parameters we pass in.
# It does not take an Omega_Lambda parameter; instead, it takes Omega_Matter, and Omega_K (for "curvature"), where
# Omega_K = 1. - Omatter - Olambda.  We will also pass in "Tcmb=0", which tells it to ignore the effects of radiation.

universe = cosmology(OmegaK=0.1, OmegaM=0.4, Tcmb=0)
@show universe
@show universe.Ω_Λ;

In [ ]:
# We can then pass that "universe" object to other functions to compute things about it.  Basically the only one you'll
# need is this `distance_modulus`, which tell you, in _magnitudes_, how much fainter an object is at the given redshift,
# versus how faint it would be if it were 10 parsecs away.

function distance_modulus(universe, z)
    DL = luminosity_dist(universe, z)
    # DL is in Megaparsecs; the distance for absolute to observed mag is 10 pc.
    5.0 * log10.(DL.val * 1e6 / 10.0)
end;

In [ ]:
# Here's our scalar estimate of the absolute mag.
abs_mag

## Part 1 - The Log-likelihood terrain

First, you have to write out the likelihood function for the observed supernova data, given cosmological model parameters.

That is, please complete the following function.  It will be passed vectors of `z`, `mag`, and `mag_error` measurements,
plus scalar parameters `M`, `Omega_M` and `Omega_DE`.  You will need to create a "cosmology" object, find the _distance modulus_ for
each redshift `z`, and add that to the absolute mag `M` to get the _predicted_ magnitude.  You will then compare that to each
measured magnitude, and compute the likelihood.

In [ ]:
# MY CODE
function supernova_log_likelihood(z, mag, mag_error, M, OmegaM, OmegaDE)
    cosmo = cosmology(OmegaK=1 - OmegaDE - OmegaM, OmegaM=OmegaM, Tcmb=0)

    # Calculate distance modulus for each redshift
    dist_moduli = map(x -> distance_modulus(cosmo, x), z)

    # Calculate predicted magnitudes
    predicted_mags = dist_moduli .+ M

    # Calculate log likelihood
    log_likelihood = -0.5 * sum(((predicted_mags .- mag) ./ mag_error) .^ 2)

    return log_likelihood
end

# # Example call to the function
log_likelihood_value = supernova_log_likelihood(data.z, data.mag, data.sigma_mag, abs_mag, 0.29, 0.71)
println("Log-likelihood: $log_likelihood_value")
# Optimize the log-likelihood to find the best parameters



In [ ]:

resolution = 100
grid_multiplier = 3

# Initialize the matrix for storing log-likelihood values
likelihood_values = zeros(resolution * grid_multiplier, resolution * grid_multiplier)

# Calculate log-likelihoods over a grid
for omegaM_index in 1:resolution*grid_multiplier
    for omegaDE_index in 1:resolution*grid_multiplier
        try
            omegaM_value = omegaM_index / resolution
            omegaDE_value = omegaDE_index / resolution
            likelihood_values[omegaM_index, omegaDE_index] = supernova_log_likelihood(data.z, data.mag, data.sigma_mag, abs_mag, omegaM_value, omegaDE_value)
        catch
            likelihood_values[omegaM_index, omegaDE_index] = -Inf
        end
    end
end

# Define the range for the axes based on the multiplier
omegaM_range = LinRange(0, grid_multiplier, resolution * grid_multiplier)
omegaDE_range = LinRange(0, grid_multiplier, resolution * grid_multiplier)

# Plot the heatmap
heatmap(omegaM_range, omegaDE_range, likelihood_values, colorrange=[maximum(likelihood_values) - 20, maximum(likelihood_values)])


<img src="perlmutter-fig7.png" width="400"/>



This will "try" to run the `supernova_log_likelihood` function, and if it fails, it will go into the "catch" branch.

## Part 2 - Using MCMC to sample from the likelihood

Next, we will use Markov Chain Monte Carlo to draw samples from the likelihood distribution.

You can start with the `mcmc` function from the lecture.

You will need to tune the MCMC proposal's step sizes (also known as "jump sizes").  To do this, you can use
the variant of the `mcmc` routine that cycles through the parameters and only jumps one at a time, named
`mcmc_cyclic` in the updated lecture notebook.  After tuning the step sizes with `mcmc_cyclic`, you can go back
to the plain `mcmc` routine if you want, or stick with `mcmc_cyclic`; it is up to you.

Please plot the samples from your MCMC chains, to demonstrate that the chain looks like it has converged.  Ideally, you
would like to see reasonable acceptance rates, and you would like to see the samples "exploring" the parameter space.
Decide how many step you need to run the MCMC routine for, and write a sentence or two describing why you think that's
a good number.

For this part, please include the `M` (absolute magnitude) as a parameter that you are fitting -- so you are fitting for `M`
in addition to `Omega_M` and `Omega_DE`.  This is a quite standard situation where you have a "nuisance" parameter `M`
that you don't really care about, in addition to the `Omega` parameters that you do care about.

It is quite common to plot the results from an MCMC sampling using a "corner plot", which shows the distribution of
each of the individual parameters, and the joint distributions of pairs of parameters.  This will help you determine
whether some of the parameters are correlated with each other.

Below is a function you can use to generate corner plots from your chain -- call it like `cornerplot(chain, ["M", "Omega_M", "Omega_DE"])`.  There is also a CornerPlot package (https://juliapackages.com/p/cornerplot) but I have not had luck getting it
to work for me.

Once you have made you corner plots, please write a few sentences interpreting what you see.  Is the nuisance parameter `M` correlated with the Omegas?  Are the Omegas correlated with each other?

In [ ]:
# Here we need to redefine the log-likelihood function to take in a vector of parameters + write proposal function
# MY CODE
function supernova_log_likelihood_v2(M, OmegaM, OmegaDE)
    cosmo_v2 = cosmology(OmegaK=1 - OmegaDE - OmegaM, OmegaM=OmegaM, Tcmb=0)


    dist_moduli = map(x -> distance_modulus(cosmo_v2, x), z)

    dist_mod_val = 0.0
    try
        dist_mod_val = map(x -> distance_modulus(dist_moduli, x), data.z)
    catch
        dist_mod_val = -Inf
    end
    f = dist_moduli .+ M

    chi = (f .- data.mag) ./ data.sigma_mag
    log_like_2 = sum(-0.5 .* chi .^ 2)

    return log_like_2
end

# Function to propose new parameter values
function gen_new_proposal(M, OmegaM, OmegaDE, jump_size)
    return [M, OmegaM, OmegaDE] .+ randn(length([M, OmegaM, OmegaDE])) .* jump_size
end

In [ ]:
#edit mcmc function
function mcmc_cyclic(logprob_func, propose_func, M, OmegaM, OmegaDE, n_steps, jump_size)
    p = [M, OmegaM, OmegaDE]
    logprob = logprob_func(p[1], p[2], p[3])
    chain = zeros(n_steps, length(p))
    n_accept = zeros(length(p))

    for i in 1:n_steps

        # We're going to update one index at a time... 1, 2, 1, 2, ....
        update_index = 1 + ((i - 1) % length(p))

        # Call the proposal function to generate new values for all parameters...
        p_prop = propose_func(p[1], p[2], p[3], jump_size)
        # ... but then only keep one of the new parameter values!
        p_new = copy(p)
        p_new[update_index] = p_prop[update_index]

        logprob_new = logprob_func(p_new[1], p_new[2], p_new[3]) # Call the log-likelihood function with vector of parameters

        ratio = exp(logprob_new - logprob)
        if ratio > 1
            # Jump to the new place
            p = p_new
            logprob = logprob_new
            n_accept[update_index] += 1
        else
            # Jump to the new place with probability "ratio"
            u = rand()
            if u < ratio
                # Jump to the new place
                p = p_new
                logprob = logprob_new
                n_accept[update_index] += 1
            else
                # Stay where we are
            end
        end
        chain[i, 1:end] = p_new
    end
    # The number of times we step each parameter is roughly n_steps / n_parameters
    return chain, n_accept ./ (n_steps ./ length(p))
end;

In [ ]:
# now we define arbitrary number of steps 
num_steps = 10000

values, acceptance_rate = mcmc_cyclic(supernova_log_likelihood_v2, gen_new_proposal, -15.0, 0.2, 0.5, num_steps, [0.01, 0.08, 0.08])

In [ ]:
f = Figure()

# Plot for M
ax1 = Axis(f[1, 1], title="M over Steps")
lines!(ax1, LinRange(1, num_steps, num_steps), values[:, 1])

# Plot for Omega_DE
ax2 = Axis(f[2, 1], title="Omega_DE over Steps")
lines!(ax2, LinRange(1, num_steps, num_steps), values[:, 3])

# Plot for Omega_M
ax3 = Axis(f[3, 1], title="Omega_M over Steps")
lines!(ax3, LinRange(1, num_steps, num_steps), values[:, 2])

f


In [ ]:
function cornerplot(x, names; figsize=(600, 600))
    # how many columns of data
    dim = size(x, 2)
    # rows to plot
    idxs = 1:size(x, 1)
    f = Figure(size=figsize)
    for i in 1:dim, j in 1:dim
        if i < j
            continue
        end
        ax = Axis(f[i, j], aspect=1,
            topspinevisible=false,
            rightspinevisible=false,)
        if i == j
            hist!(x[idxs, i], direction=:y)
            ax.xlabel = names[i]
        else
            #scatter!(x[idxs,j], x[idxs,i], markersize=4)
            hexbin!(x[idxs, j], x[idxs, i])
            ax.xlabel = names[j]
            ax.ylabel = names[i]
        end
    end
    f
end;

In [ ]:
cornerplot(values, ["M", "Omega_M", "Omega_DE"])

Finally, please try to make a contour plot similar to Perlmutter et al.'s Figure 7.  From your MCMC chain, you can pull out the `Omega_M` and `Omega_DE` arrays, and then create a 2-d histogram.  Once you have a 2-d histogram, you can use the `contour` function to find and plot the contours in that histogram.

In [ ]:
n_bins = (100, 100)
histogram_data = FHist.Hist2D((values[:, 2], values[:, 3]), nbins=n_bins)

# Get the counts and the center points of the bins
bin_counts = bincounts(histogram_data)
bin_centers_x, bin_centers_y = bincenters(histogram_data)

# Create a contour plot using the histogram data
contour_figure = contour(bin_centers_x, bin_centers_y, bin_counts)

contour_figure

#size issue